## Startup

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import norm
import scipy.stats as scs

import os
import pickle
pd.set_option("display.precision", 3)

In [27]:
np.random.seed(42)

In [28]:
from scripts.params import get_params

params = get_params()

In [29]:
dataroute = params["dataroute"]
resultsroute = params["resultsroute"]
dumproute = params["dumproute"]
graphsroute = params["graphsroute"]
descriptivegraphsroute=params["descriptivegraphsroute"]

## Data Retrieval

In [30]:
name = f'finaldf_train_{params["tablename"]}.pickle'
filename = os.path.join(dataroute, name)
with open(filename, "rb") as handle:
    df = pickle.load(handle)

## Descriptive graphs

In [31]:
log_rets_list=[]
vol_list=[]
for column in df.columns: 
    if column.endswith("log_rets"):
        log_rets_list.append(column)
    if column.endswith("gk_vol"):
        vol_list.append(column)

In [32]:
sb.set_style(style='darkgrid')
sb.set_palette(sb.color_palette(palette='deep'))

In [33]:
for column in log_rets_list:
    fig=df[column].plot(title=column
                        ).get_figure()
    fig.savefig(os.path.join(descriptivegraphsroute, f"{column}_log_rets.png"))
    plt.close()

In [34]:
for column in vol_list:
    fig=df[column].plot(title=column).get_figure() 
    fig.savefig(os.path.join(descriptivegraphsroute, f"{column}_gk_vol.png"))
    plt.close()

In [35]:
for column in vol_list:
    fig=df[column].plot(title=column, 
                        ylim=(0, 0.225) # these limits are set so as that all are in same scale as arg
                        ).get_figure() 
    fig.savefig(os.path.join(descriptivegraphsroute, f"{column}_gk_vol_AR_comparison.png"))
    plt.close()

### Autocorrelograms

In [36]:
acf_lags=90

In [37]:
def save_acf(column, path):
    fig = plt.figure()
    ax1 = fig.add_subplot(1, 1, 1)
    plot_acf(df[column],
                lags=acf_lags,                 # Cuantos lags busco autocorrelacionar
                zero=False,                # Si tomo el lag cero
                alpha=0.05,                # El rango de certeza marcado en azul
                use_vlines=False,          # Lineas verticales que conectan cada punto con el eje x
                ax=ax1)                    # La posicion en la figura
    ax1.grid(True)
    ax1.set_xlabel('Lag')
    ax1.set_ylabel('Autocorrelation')
    ax1.set_title(f'Autocorrelation of {column}')
    fig.savefig(path)
    plt.close()


In [38]:
for column in log_rets_list:
    save_acf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 f"{column}_acf_log_rets.png"))

In [39]:
for column in vol_list:
    save_acf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 f"{column}_acf_gk_vol.png"))

In [40]:
def save_pacf(column, path):
    """
    """
    fig = plt.figure(figsize=[13.6, 10.2])
    ax1 = fig.add_subplot(1, 1, 1)
    plot_pacf(df[column],
                method='ywa',                 # Metodo de Yule Walker con correccion de sesgo por autocovarianzas
                lags=acf_lags,                # Cuantos lags busco autocorrelacionar
                zero=False,                   # Si tomo el lag cero
                alpha=0.05,                   # El rango de certeza marcado en azul
                use_vlines=False,             # Lineas verticales que conectan cada punto con el eje x
                ax=ax1)                       # La posicion en la figura
    ax1.grid(True)
    ax1.set_xlabel('Lag')
    ax1.set_ylabel('Partial Autocorrelation')
    ax1.set_title(f'Partial Autocorrelation of {column}')

    fig.savefig(path)
    plt.close()

In [41]:
for column in log_rets_list:
    save_pacf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 f"{column}_pacf_log_rets.png"))

In [42]:
for column in vol_list:
    save_acf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 f"{column}_pacf_gk_vol.png"))

In [43]:
def save_hist_normal(column, path):
    fig = plt.figure(figsize=[13.6, 5.1])
    ax1 = fig.add_subplot(1, 1, 1)

    sb.distplot(df[column].fillna(0),
                ax=ax1,
                hist=True,
                bins=int(np.ceil(np.log2(len(df[column].index)) + 15)),
                label='Observed data KDE',
                fit=norm,
                fit_kws = {"color":"r", "lw":3, "label":"Fitted Normal"})
    plt.legend()
    plt.grid(True)
    plt.xlabel('Log Returns')
    plt.ylabel('Frequency')
    plt.title(f'Histogram for return frequency for {column}')
    fig.savefig(path)
    plt.close()

In [44]:
for column in log_rets_list:
    save_hist_normal(
        column,
        os.path.join(
            descriptivegraphsroute, 
            f"{column}_histogram.png"))

C:\Users\alfredo.sampron\AppData\Local\Temp\ipykernel_36752\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users\alfredo.sampron\AppData\Local\Temp\ipykernel_36752\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users

In [45]:
for column in vol_list:
    save_hist_normal(
        column,
        os.path.join(
            descriptivegraphsroute, 
            f"{column}_vol_histogram.png"))

C:\Users\alfredo.sampron\AppData\Local\Temp\ipykernel_36752\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users\alfredo.sampron\AppData\Local\Temp\ipykernel_36752\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users

In [46]:
def analyze_skew_kurt(dataframe):
    results = pd.DataFrame(index=dataframe.columns, 
                           columns=['Arithmetic Mean', 'Standard Deviation', 'Skewness', 'Skewness P-Value', 'Kurtosis', 'Kurtosis P-Value', 'Normaltest Stat', 'Normaltest P-Value'])
    for column in dataframe.columns:
        mean = np.mean(dataframe[column])
        std = np.std(dataframe[column])
        skew_val, skew_pval = scs.skewtest(dataframe[column], nan_policy='omit')
        kurt_val, kurt_pval = scs.kurtosistest(dataframe[column], nan_policy='omit')
        normtest_stat, normtest_pval = scs.normaltest(dataframe[column], nan_policy='omit')
        results.loc[column] = [mean, std, skew_val, skew_pval, kurt_val, kurt_pval, normtest_stat, normtest_pval]

    return results

In [47]:
log_rets_list

['^MERV_log_rets',
 'GGAL.BA_log_rets',
 'GGAL_log_rets',
 'YPFD.BA_log_rets',
 'YPF_log_rets',
 'EDN.BA_log_rets',
 'EDN_log_rets',
 'BMA.BA_log_rets',
 'BMA_log_rets',
 'BBAR.BA_log_rets',
 'BBAR_log_rets',
 'USD_log_rets',
 'MERV_FX_log_rets']

In [48]:
desc_table = analyze_skew_kurt(df[log_rets_list].fillna(0))
desc_table

,Arithmetic Mean,Standard Deviation,Skewness,Skewness P-Value,Kurtosis,Kurtosis P-Value,Normaltest Stat,Normaltest P-Value
^MERV_log_rets,0.002,0.025,-33.568,0.0,29.397,0.0,1990.997,0.0
GGAL.BA_log_rets,0.002,0.035,-31.284,0.0,29.158,0.0,1828.873,0.0
GGAL_log_rets,0.0,0.037,-39.965,0.0,31.665,0.0,2599.846,0.0
YPFD.BA_log_rets,0.002,0.032,-10.451,0.0,19.293,0.0,481.454,0.0
YPF_log_rets,0.0,0.033,-21.903,0.0,24.661,0.0,1087.915,0.0
EDN.BA_log_rets,0.002,0.038,-27.399,0.0,28.678,0.0,1573.133,0.0
EDN_log_rets,0.0,0.042,-36.205,0.0,31.096,0.0,2277.769,0.0
BMA.BA_log_rets,0.002,0.036,-28.002,0.0,28.137,0.0,1575.8,0.0
BMA_log_rets,0.0,0.037,-34.883,0.0,30.428,0.0,2142.73,0.0
BBAR.BA_log_rets,0.002,0.036,-28.406,0.0,28.275,0.0,1606.41,0.0


In [49]:
desc_table.to_csv(os.path.join(resultsroute, "descriptive_table.csv"))

In [50]:
desc_table.to_clipboard()